### Planning with task decomposition

In this notebook, we will look at several methods of planning that allow you to decompose a complex task into smaller pieces.

Methods that we will cover include

1. Chain of Thought prompting (COT)
2. Tree of Thought prompting (TOT)
3. Plan and Solve

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") ## Put your OpenAI API key here
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY") ## Put your Tavily Search API key here
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") ## Put your Langsmith API key here
os.environ["LANGCHAIN_HUB_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") ## Put your Langsmith API key here
os.environ["LANGCHAIN_TRACING_V2"] = 'true' ## Set this as True
os.environ["LANGCHAIN_ENDPOINT"] = 'https://api.smith.langchain.com/' ## Set this as: https://api.smith.langchain.com/
os.environ["LANGCHAIN_HUB_API_URL"] = 'https://api.hub.langchain.com' ## Set this as : https://api.hub.langchain.com
os.environ["LANGCHAIN_PROJECT"] = 'llm-agents-planning'

## Chain of Thought Prompting

Learn More: https://deepgram.com/learn/chain-of-thought-prompting-guide

![COT](images/cot.jpeg)

In [3]:
open_source_model = 'mistralai/Mistral-7B-Instruct-v0.3'

In [4]:
## Add Together API Key
os.environ["TOGETHER_API_KEY"] = os.getenv("TOGETHER_API_KEY")
### import together
from together_llm import TogetherLLM
## Langchain imports
from langchain_openai import OpenAI


In [5]:
llm = TogetherLLM(model=open_source_model, temperature=0.3, max_tokens=800)

In [6]:
from langchain.prompts import PromptTemplate


# Template for the problem and the desired format of the answer
template = """Solve the following problem step by step. First give the answer and then the steps.

{problem}
"""
prompt = PromptTemplate(template=template, input_variables=["problem"])

cot_chain = prompt | llm
math_problem = '''
A train leaves City A traveling towards City B at a speed of 120 km/h.
 Two hours later, a slower train leaves City B traveling towards City A at a speed of 80 km/h.
   If the distance between the two cities is 960 km, how far from City A will the two trains meet?
'''

# Use the chain to generate the result
result = cot_chain.invoke({"problem": math_problem})
print(result)


 The two trains will meet 640 km away from City A.

Here are the steps to solve the problem:

1. Let x be the distance the faster train travels in the two hours before the slower train starts moving.
2. In those two hours, the slower train remains stationary, so the combined distance the faster train travels is equal to the distance between the cities minus the distance the slower train needs to travel to meet the faster train.
3. So, we have the equation: 120t + 120(2) = 960 - 80(t - 2), where t is the time in hours it takes for the trains to meet after both start moving.
4. Simplifying the equation, we get: 120t + 240 = 960 - 80t + 160.
5. Combining like terms, we get: 240 + 80t = 960 - 120t.
6. Solving for t, we get: t = (960 - 240) / (120 + 80) = 720 / 200 = 3.6 hours.
7. Now that we know the time it takes for the trains to meet, we can find the distance the faster train travels before the slower train starts moving: 120 * 3.6 = 432 km.
8. Since the trains meet after traveling an a

### Chain of Thought  with Self Consistency
- Run the problem several times and take the most common answer

![COT-SC](images/cot-sc.jpg)

### COT with Examples:

cot_prompt = '''Solve the following problem step-by-step, clearly showing your calculations and reasoning.
Problem: {math_problem}

Example of a math problem solved step by step

"Solve the following problem step-by-step, clearly showing your calculations and reasoning.
Problem: A group of friends went to a restaurant. The total bill was $120, and they wanted to leave a 15% tip. How much should each person pay if there were 5 friends splitting the bill and tip evenly?"

Response:

Step 1: Calculate the tip amount.

15% of the $120 bill is (15/100) * $120 = $18.
Step 2: Calculate the total cost including the tip.

The bill plus tip is $120 + $18 = $138.
Step 3: Divide the total cost by the number of friends.

Each friend should pay $138 / 5 friends = $27.60.
Therefore, each of the 5 friends should pay $27.60 to cover the bill and tip evenly..'''

## Plan and Solve

Paper: https://arxiv.org/pdf/2305.04091v3

![Plan_Solve](images/plan-solve.png)

In [7]:
# Template for the problem and the desired format of the answer
template = """Solve the following problem by first devising a step by step plan for solving the problem. Then carry out the plan step by step

{problem}
"""
prompt = PromptTemplate(template=template, input_variables=["problem"])

plan_solve_chain = prompt | llm
math_problem = '''
A train leaves City A traveling towards City B at a speed of 120 km/h.
 Two hours later, a slower train leaves City B traveling towards City A at a speed of 80 km/h.
   If the distance between the two cities is 960 km, how far from City A will the two trains meet?
'''

# Use the chain to generate the result
result = plan_solve_chain.invoke({"problem": math_problem})
print(result)

 Step 1: Understand the problem
- We have two trains traveling towards each other.
- Train A leaves City A at a speed of 120 km/h.
- Train B leaves City B (which is 960 km away from City A) at a speed of 80 km/h, 2 hours after Train A has left.
- We need to find out how far from City A the two trains will meet.

Step 2: Convert the time difference
- Since Train B leaves 2 hours after Train A, we can say that Train A has traveled for 2 hours before Train B starts moving.

Step 3: Calculate the distance covered by Train A in 2 hours
- Distance = Speed * Time
- Distance covered by Train A = 120 km/h * 2 hours = 240 km

Step 4: Calculate the combined speed of both trains
- Combined speed = Speed of Train A + Speed of Train B = 120 km/h + 80 km/h = 200 km/h

Step 5: Calculate the remaining distance between the trains
- Remaining distance = Total distance - Distance covered by Train A = 960 km - 240 km = 720 km

Step 6: Calculate the time it takes for the trains to meet
- Time = Remaining di

## Tree of Thought

Read more about it here: https://www.promptingguide.ai/techniques/tot

![TOT](images/tot.png)

In [8]:
# set the LANGCHAIN_API_KEY environment variable (create key in settings)
from langchain import hub

tot_step1 = hub.pull("rachnogstyle/nlw_jan24_cot_step1")
tot_step2 = hub.pull("rachnogstyle/nlw_jan24_cot_step2")
tot_step3 = hub.pull("rachnogstyle/nlw_jan24_cot_step3")
tot_step4 = hub.pull("rachnogstyle/nlw_jan24_cot_step4")

print(tot_step1.template)
print(tot_step2.template)
print(tot_step3.template)
print(tot_step4.template)

Step1 :
 
I have a problem related to {input}. Could you brainstorm three distinct solutions? Please consider a variety of factors such as {perfect_factors}
A:
Step 2:

For each of the three proposed solutions, evaluate their potential. Consider their pros and cons, initial effort needed, implementation difficulty, potential challenges, and the expected outcomes. Assign a probability of success and a confidence level to each option based on these factors

{solutions}

A:
Step 3:

For each solution, deepen the thought process. Generate potential scenarios, strategies for implementation, any necessary partnerships or resources, and how potential obstacles might be overcome. Also, consider any potential unexpected outcomes and how they might be handled.

{review}

A:
Step 4:

Based on the evaluations and scenarios, rank the solutions in order of promise. Provide a justification for each ranking and offer any final thoughts or considerations for each solution
{deepen_thought_process}

A:


In [11]:
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough


llm_openai = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2)

In [15]:

prompt1 = PromptTemplate(template=tot_step1.template, input_variables=["input", "perfect_factors"])
prompt2 = PromptTemplate(template=tot_step2.template, input_variables=["solutions"])
prompt3 = PromptTemplate(template=tot_step3.template, input_variables=["review"])
prompt4 = PromptTemplate(template=tot_step4.template, input_variables=["deepen_thought_process"])

tot_complete_chain = (
        prompt1
        | llm_openai
        | {'solutions': RunnablePassthrough()}
        | prompt2
        | llm_openai
        | {'review': RunnablePassthrough()}
        | prompt3
        | llm_openai
        | {'deepen_thought_process': RunnablePassthrough()}
        | prompt4
        | llm_openai
        | {'ranked_solution': RunnablePassthrough()}
)

In [16]:
results = tot_complete_chain.invoke( {
                "input": "human colonization of Mars",
                "perfect_factors": "The distance between Earth and Mars is very large, making regular resupply difficult"
            })
print(results)

{'ranked_solution': AIMessage(content='Ranking of Solutions in Order of Promise:\n\n1. Option 3: Establishing partnerships with other space agencies or private companies\n2. Option 1: Developing sustainable habitats on Mars\n3. Option 2: Creating a reliable transportation system between Earth and Mars\n\nJustification for Rankings:\n\nOption 3 is ranked the highest as it offers the most promising path forward for Mars colonization by leveraging the expertise and resources of multiple organizations. Collaboration with other space agencies and private companies can increase the likelihood of success and maximize shared knowledge and support.\n\nOption 1 is ranked second as developing sustainable habitats on Mars is crucial for long-term colonization. While it presents challenges such as construction and maintenance, the use of advanced technologies and partnerships with experts can overcome these obstacles.\n\nOption 2 is ranked last as creating a reliable transportation system, while im

In [18]:
print(results['ranked_solution'].content)

Ranking of Solutions in Order of Promise:

1. Option 3: Establishing partnerships with other space agencies or private companies
2. Option 1: Developing sustainable habitats on Mars
3. Option 2: Creating a reliable transportation system between Earth and Mars

Justification for Rankings:

Option 3 is ranked the highest as it offers the most promising path forward for Mars colonization by leveraging the expertise and resources of multiple organizations. Collaboration with other space agencies and private companies can increase the likelihood of success and maximize shared knowledge and support.

Option 1 is ranked second as developing sustainable habitats on Mars is crucial for long-term colonization. While it presents challenges such as construction and maintenance, the use of advanced technologies and partnerships with experts can overcome these obstacles.

Option 2 is ranked last as creating a reliable transportation system, while important, may not be as immediately critical as esta